# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
city_weather_df = pd.read_csv("./output/cities_weather.csv")

city_weather_df


,City ID,City,Clouds,Country,Humidity,Lat,Lng,Temperature,Wind Speed
0,4030556,Rikitea,10,PF,75,-23.1203,-134.9692,74.16,14.50
1,6145890,Shelburne,83,CA,54,44.0787,-80.2041,63.21,5.86
2,6167817,Torbay,40,CA,80,47.6666,-52.7314,65.16,17.27
3,3137469,Sørland,98,NO,84,67.6670,12.6934,54.57,9.08
4,3466704,Castro,100,BR,94,-24.7911,-50.0119,48.25,5.61
...,...,...,...,...,...,...,...,...,...
552,2655288,Boddam,100,GB,87,57.4667,-1.7833,59.38,20.36
553,1622318,Waingapu,43,ID,56,-9.6567,120.2641,85.39,19.86
554,217570,Buta,100,CD,99,2.7858,24.7300,69.66,2.33
555,3527795,Escárcega,68,MX,79,18.6089,-90.7454,76.87,2.33


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
#configuring gmaps 

gmaps.configure(api_key=g_key)


In [9]:
#coverting the geocoordinates

locations_df = round(city_weather_df[["Lat", "Lng"]].astype(float),2)

humidity_df = city_weather_df["Humidity"].astype(float)

#locations_df
#humidity_df

In [28]:
#adding heatmaps

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations_df, weights=humidity_df, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
fit_cities = pd.DataFrame()

for index, row in city_weather_df.iterrows():
    if row["Temperature"]>70 and row["Temperature"]<80 and row["Wind Speed"]<10 and row["Clouds"]==0:
        fit_cities[index]=row
#         else:
#             fit_cities

fit_cities = fit_cities.dropna().T
fit_cities

,City ID,City,Clouds,Country,Humidity,Lat,Lng,Temperature,Wind Speed
101,101312,Turaif,0,SA,35,31.6725,38.6637,75.45,4.61
108,293842,Kiryat Gat,0,IL,52,31.61,34.7642,76.86,1.32
205,2520645,Cabra,0,ES,36,37.4725,-4.4421,74.43,1.72
220,1526273,Nur-Sultan,0,KZ,43,51.1801,71.446,73.35,6.71
246,2521847,Almuñécar,0,ES,50,36.7339,-3.6907,78.04,3.6
256,6255012,Flinders,0,AU,45,-34.5833,150.8552,71.46,5.01
257,140918,Behshahr,0,IR,39,36.6923,53.5526,75.15,3.65
308,3581194,San Ignacio,0,BZ,94,17.1561,-89.0714,77.14,0.0
432,2036892,Hohhot,0,CN,41,40.8106,111.6522,75.49,4.47
460,751335,Bafra,0,TR,56,41.5678,35.9069,77.83,5.5


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
#creating variable named hotel_df

hotel_df = fit_cities.loc[:,["City", "Country", "Lat", "Lng"]]

#adding "Hotel Name" column
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
101,Turaif,SA,31.6725,38.6637,
108,Kiryat Gat,IL,31.61,34.7642,
205,Cabra,ES,37.4725,-4.4421,
220,Nur-Sultan,KZ,51.1801,71.446,
246,Almuñécar,ES,36.7339,-3.6907,
256,Flinders,AU,-34.5833,150.8552,
257,Behshahr,IR,36.6923,53.5526,
308,San Ignacio,BZ,17.1561,-89.0714,
432,Hohhot,CN,40.8106,111.6522,
460,Bafra,TR,41.5678,35.9069,


In [19]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [23]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Nearest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field... skip.....")
        
    print("------------")
    
    # Wait 2 sec to make another api request to avoid SSL Error
    #time.sleep(2)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 101: Turaif.
Nearest hotel in Turaif is Golden Dune Hotel.
------------
Retrieving Results for Index 108: Kiryat Gat.
Nearest hotel in Kiryat Gat is Desert Gat.
------------
Retrieving Results for Index 205: Cabra.
Nearest hotel in Cabra is MS Fuente Las Piedras.
------------
Retrieving Results for Index 220: Nur-Sultan.
Nearest hotel in Nur-Sultan is The St. Regis Astana.
------------
Retrieving Results for Index 246: Almuñécar.
Nearest hotel in Almuñécar is Playacálida Spa Hotel.
------------
Retrieving Results for Index 256: Flinders.
Nearest hotel in Flinders is Lakeview Hotel Motel.
------------
Retrieving Results for Index 257: Behshahr.
Nearest hotel in Behshahr is Ashraf Hotel.
------------
Retrieving Results for Index 308: San Ignacio.
Nearest hotel in San Ignacio is San Ignacio Resort Hotel.
------------
Retrieving Results for Index 432: Hohhot.
Nearest hotel in Hohhot is Shangri-La Hotel, Hohhot.
------------
Retrieving Results for Index 460: Baf

In [24]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
101,Turaif,SA,31.6725,38.6637,Golden Dune Hotel
108,Kiryat Gat,IL,31.61,34.7642,Desert Gat
205,Cabra,ES,37.4725,-4.4421,MS Fuente Las Piedras
220,Nur-Sultan,KZ,51.1801,71.446,The St. Regis Astana
246,Almuñécar,ES,36.7339,-3.6907,Playacálida Spa Hotel
256,Flinders,AU,-34.5833,150.8552,Lakeview Hotel Motel
257,Behshahr,IR,36.6923,53.5526,Ashraf Hotel
308,San Ignacio,BZ,17.1561,-89.0714,San Ignacio Resort Hotel
432,Hohhot,CN,40.8106,111.6522,"Shangri-La Hotel, Hohhot"
460,Bafra,TR,41.5678,35.9069,Sevgi Otel


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)


# Display figure
fig


Figure(layout=FigureLayout(height='420px'))